In [20]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [21]:
import os
import http.client
import json

In [22]:
# pick a model
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [23]:
from langchain_core.tools import tool

@tool
def browse(txt: str) -> str:
    """ Call this tool if you want to search the web for current information"""
    
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": txt,
    "gl": "in"
    })
    headers = {
    'X-API-KEY': os.environ.get("SERPER_API_KEY"),
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    #print(data.decode("utf-8"))
    data_dict = json.loads(data.decode("utf-8"))
    return data_dict['organic'][0]['snippet']


In [24]:
tools_list = [browse]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [25]:
# we create a tool calling Agent by binding a list of tools to the llm
llm_with_tools = llm.bind_tools(tools_list)

In [26]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# This will store all converation
chat_history = [
    SystemMessage(content="You are a chatbot. Depending on the conversation, if user demands current information, you must use the tool 'browse' to get the information. If you do not need current information, you can answer directly. Always try to keep the conversation short and crisp.")
    ]

In [27]:
chat_history=[]

In [28]:
chat_history.append(HumanMessage(content="Tell me about the current Prime Minister of India."))

response = llm_with_tools.invoke(chat_history)
response


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8tchqqwx2', 'function': {'arguments': '{"txt":"current Prime Minister of India"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 234, 'total_tokens': 252, 'completion_time': 0.051370209, 'prompt_time': 0.019843022, 'queue_time': 0.057027368, 'total_time': 0.071213231}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b678fee8-321e-4dae-b9cd-2ffcf5696f2c-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'current Prime Minister of India'}, 'id': '8tchqqwx2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 234, 'output_tokens': 18, 'total_tokens': 252})

In [29]:
chat_history.append(response)
chat_history


[HumanMessage(content='Tell me about the current Prime Minister of India.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8tchqqwx2', 'function': {'arguments': '{"txt":"current Prime Minister of India"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 234, 'total_tokens': 252, 'completion_time': 0.051370209, 'prompt_time': 0.019843022, 'queue_time': 0.057027368, 'total_time': 0.071213231}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b678fee8-321e-4dae-b9cd-2ffcf5696f2c-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'current Prime Minister of India'}, 'id': '8tchqqwx2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 234, 'output_tokens': 18, 'total_tokens': 252})]

In [30]:
# Since there was a tool call, execute the tool and append the tool output in the converation

chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[HumanMessage(content='Tell me about the current Prime Minister of India.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8tchqqwx2', 'function': {'arguments': '{"txt":"current Prime Minister of India"}', 'name': 'browse'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 234, 'total_tokens': 252, 'completion_time': 0.051370209, 'prompt_time': 0.019843022, 'queue_time': 0.057027368, 'total_time': 0.071213231}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b678fee8-321e-4dae-b9cd-2ffcf5696f2c-0', tool_calls=[{'name': 'browse', 'args': {'txt': 'current Prime Minister of India'}, 'id': '8tchqqwx2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 234, 'output_tokens': 18, 'total_tokens': 252}),
 ToolMessage(content="Shri Narendra Modi was sworn-in as

In [31]:
chat_history.append(HumanMessage(content="What is 10 plus 10?"))

response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='20', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 311, 'total_tokens': 313, 'completion_time': 0.015686921, 'prompt_time': 0.025857881, 'queue_time': 0.051507649, 'total_time': 0.041544802}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4aa49656-1bd1-4447-82bf-622eb801d7d1-0', usage_metadata={'input_tokens': 311, 'output_tokens': 2, 'total_tokens': 313})